In [4]:
import ee
#ee.Authenticate()
ee.Initialize()

In [5]:
import pandas as pd
import numpy as np
import time 

In [6]:
locations_filepath="data/yield_data.csv"
collection_id="MODIS/061/MOD09A1"
imgcoll = ee.ImageCollection(collection_id) \
    .filterBounds(ee.Geometry.Rectangle(-106.5, 50, -64, 23)) \
    .filterDate("2002-12-31", "2012-8-4")
# Get the number of images.
count = imgcoll.size()
print('Count: ', str(count.getInfo())+'\n')

Count:  441



In [7]:
infos = imgcoll.getInfo()
dates = [i['properties']['system:index'] for i in infos['features']]
print(len(dates))
dates[:46]

441


['2003_01_01',
 '2003_01_09',
 '2003_01_17',
 '2003_01_25',
 '2003_02_02',
 '2003_02_10',
 '2003_02_18',
 '2003_02_26',
 '2003_03_06',
 '2003_03_14',
 '2003_03_22',
 '2003_03_30',
 '2003_04_07',
 '2003_04_15',
 '2003_04_23',
 '2003_05_01',
 '2003_05_09',
 '2003_05_17',
 '2003_05_25',
 '2003_06_02',
 '2003_06_10',
 '2003_06_18',
 '2003_06_26',
 '2003_07_04',
 '2003_07_12',
 '2003_07_20',
 '2003_07_28',
 '2003_08_05',
 '2003_08_13',
 '2003_08_21',
 '2003_08_29',
 '2003_09_06',
 '2003_09_14',
 '2003_09_22',
 '2003_09_30',
 '2003_10_08',
 '2003_10_16',
 '2003_10_24',
 '2003_11_01',
 '2003_11_09',
 '2003_11_17',
 '2003_11_25',
 '2003_12_03',
 '2003_12_11',
 '2003_12_19',
 '2003_12_27']

In [1]:
dates[:]

NameError: name 'dates' is not defined

In [95]:
x = imgcoll.first()

In [99]:
x.bandNames().getInfo()

['sur_refl_b01',
 'sur_refl_b02',
 'sur_refl_b03',
 'sur_refl_b04',
 'sur_refl_b05',
 'sur_refl_b06',
 'sur_refl_b07',
 'QA',
 'SolarZenith',
 'ViewZenith',
 'RelativeAzimuth',
 'StateQA',
 'DayOfYear']

In [105]:
x.select(0).bandNames().getInfo()

['sur_refl_b01']

In [7]:
def _append_im_band(current, previous):
    # Transforms an Image Collection with 1 band per Image into a single Image with items as bands
    # Author: Jamie Vleeshouwer

    # Rename the band
    previous = ee.Image(previous)
    current = current.select([0, 1, 2, 3, 4, 5, 6])
    # Append it to the result (Note: only return current item on first element/iteration)
    return ee.Algorithms.If(
        ee.Algorithms.IsEqual(previous, None),
        current,
        previous.addBands(ee.Image(current)),
    )

In [8]:
img = imgcoll.iterate(_append_im_band)
img = ee.Image(img)
print(len(img.getInfo()['bands']))
print(count.getInfo()*7)

3087
3087


In [9]:
region = ee.FeatureCollection("TIGER/2018/Counties")
# turn the strings into numbers, see
# https://developers.google.com/earth-engine/datasets/catalog/TIGER_2018_Counties
def county_to_int(feature):
    return feature.set("COUNTYFP", ee.Number.parse(feature.get("COUNTYFP")))

def state_to_int(feature):
    return feature.set("STATEFP", ee.Number.parse(feature.get("STATEFP")))

region = region.map(county_to_int)
region = region.map(state_to_int)

In [15]:
df = pd.read_csv(locations_filepath)
county_data = np.unique(df[["State ANSI", "County ANSI"]].values, axis=0)

In [16]:
folder_name = 'GEE_Resources_Raw'
scale=500
projection = img.select(['sur_refl_b01', 'sur_refl_b04', 'sur_refl_b03']).projection().getInfo()
projection

{'type': 'Projection',
 'crs': 'SR-ORG:6974',
 'transform': [463.3127165279165,
  0,
  -20015109.354,
  0,
  -463.3127165279167,
  7783653.637669001]}

In [20]:
info = img.getInfo()

In [23]:
info['properties']

{'system:time_start': 1041379200000,
 'google:max_source_file_timestamp': 1626507615000,
 'system:footprint': {'type': 'LinearRing',
  'coordinates': [[-180, -90],
   [180, -90],
   [180, 90],
   [-180, 90],
   [-180, -90]]},
 'system:time_end': 1042070400000,
 'system:asset_size': 25859198972,
 'system:index': '2003_01_01'}

In [19]:
for state_id, county_id in county_data:
    fname = "{}_{}".format(int(state_id), int(county_id))
    file_region = region.filterMetadata(
        "COUNTYFP", "equals", int(county_id)
        ).filterMetadata("STATEFP", "equals", int(state_id))
    
    file_region = ee.Feature(file_region.first())
    processed_img = img.clip(file_region)
    #file_region = None
    
    # Export the image, specifying scale and region.
    task = ee.batch.Export.image.toDrive(**{
        'image': processed_img.toFloat(),
        'description': fname,
        'folder':folder_name,   ## it will create a folder an dump the image there 
        #'scale': scale,
        #'fileNamePrefix':"{}/{}".format(prefix,fname),
        #'crs': test_crs, #projection['crs'],
        #'crsTransform': projection['transform'],
        'region': file_region.geometry(),
        'fileFormat': 'GeoTIFF',
        'maxPixels':1e13,
    })
    task.start()

    while task.active():
      print('Working {},  Polling for task (id: {}).'.format(fnamem task.id))
      time.sleep(120)

    print(task.status())
    

Working XFQQL6HTKP7UWH6SJ743CQ2J,  Polling for task (id: 1_1).
Working XFQQL6HTKP7UWH6SJ743CQ2J,  Polling for task (id: 1_1).
Working XFQQL6HTKP7UWH6SJ743CQ2J,  Polling for task (id: 1_1).
Working XFQQL6HTKP7UWH6SJ743CQ2J,  Polling for task (id: 1_1).
Working XFQQL6HTKP7UWH6SJ743CQ2J,  Polling for task (id: 1_1).
Working XFQQL6HTKP7UWH6SJ743CQ2J,  Polling for task (id: 1_1).
Working XFQQL6HTKP7UWH6SJ743CQ2J,  Polling for task (id: 1_1).
Working XFQQL6HTKP7UWH6SJ743CQ2J,  Polling for task (id: 1_1).
Working XFQQL6HTKP7UWH6SJ743CQ2J,  Polling for task (id: 1_1).
Working XFQQL6HTKP7UWH6SJ743CQ2J,  Polling for task (id: 1_1).
Working XFQQL6HTKP7UWH6SJ743CQ2J,  Polling for task (id: 1_1).
Working XFQQL6HTKP7UWH6SJ743CQ2J,  Polling for task (id: 1_1).
Working XFQQL6HTKP7UWH6SJ743CQ2J,  Polling for task (id: 1_1).
Working XFQQL6HTKP7UWH6SJ743CQ2J,  Polling for task (id: 1_1).
{'state': 'COMPLETED', 'description': '1_1', 'creation_timestamp_ms': 1651634608437, 'update_timestamp_ms': 1651635398

ValueError: cannot convert float NaN to integer